In [28]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(".."))
from scripts.sa2_names import get_sa2_names
from scripts.preproccessing import add_data
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [2]:
SA2_names = get_sa2_names()

In [3]:
SA2_names

['Alfredton',
 'Ballarat',
 'Buninyong',
 'Delacombe',
 'Smythes Creek',
 'Wendouree - Miners Rest',
 'Ballarat East - Warrenheip',
 'Ballarat North - Invermay',
 'Canadian - Mount Clear',
 'Sebastopol - Redan',
 'Bacchus Marsh Surrounds',
 'Creswick - Clunes',
 'Daylesford',
 'Gordon (Vic.)',
 'Avoca',
 'Beaufort',
 'Golden Plains - North',
 'Maryborough (Vic.)',
 'Maryborough Surrounds',
 'Bendigo',
 'California Gully - Eaglehawk',
 'East Bendigo - Kennington',
 'Flora Hill - Spring Gully',
 'Kangaroo Flat - Golden Square',
 'Maiden Gully',
 'Strathfieldsaye',
 'White Hills - Ascot',
 'Bendigo Surrounds - South',
 'Castlemaine',
 'Castlemaine Surrounds',
 'Heathcote',
 'Kyneton',
 'Woodend',
 'Bendigo Surrounds - North',
 'Loddon',
 'Bannockburn',
 'Golden Plains - South',
 'Winchelsea',
 'Belmont',
 'Geelong',
 'Geelong West - Hamlyn Heights',
 'Highton',
 'Lara',
 'Leopold',
 'Newcomb - Moolap',
 'Newtown (Vic.)',
 'North Geelong - Bell Park',
 'Charlemont',
 'Corio - Lovely Banks'

In [4]:
years = [str(year) for year in range(2006, 2025)]

# Create a list that repeats each SA2 name for every year
sa2_column = SA2_names * len(years)

# Create a list that repeats each year for all SA2 names
year_column = years * len(SA2_names)

# Create the DataFrame
df = pd.DataFrame({
    'SA2_NAME21': sa2_column,
    'year': year_column
})

In [5]:
ndf = add_data(df)

In [9]:
nndf = ndf.drop(['SA2_NAME21', 'year', 'housing_index', 'cpi_without_housing'], axis=1)

In [10]:
nndf

,num_homeless_persons,avg_household_size,num_businesses,median_income,median_age,median_weekly_rent,percent_aboriginal_torres_strait_islander,percent_au_citizen,percent_overseas_born,percent_rental_properties,population,percent_unemployed
0,36.078873,2.800000,0.0,34692.8,28.02,190.333333,0.533333,4.866667,5.50,16.133333,10464.086957,4.833333
1,36.735775,2.356667,2041.2,38950.5,33.34,192.966667,0.496667,5.593333,7.42,33.866667,10613.030691,6.523333
2,0.000000,2.846667,200.8,34089.8,30.64,240.666667,0.433333,4.266667,11.14,13.893333,10779.879795,6.293333
3,31.000000,2.703333,0.0,31742.1,32.94,157.833333,1.206667,6.453333,2.74,19.370000,10965.838875,5.680000
4,0.000000,3.000000,200.7,35227.8,37.04,217.333333,0.900000,6.140000,7.26,4.020000,11096.473146,4.440000
...,...,...,...,...,...,...,...,...,...,...,...,...
9913,20.200000,2.093333,504.1,36575.4,52.34,279.000000,1.313333,10.200000,15.68,20.053333,12834.097187,3.906667
9914,22.833333,2.516667,1124.8,49385.9,42.01,200.833333,1.866667,6.883333,6.90,15.750000,12696.762148,2.883333
9915,10.133333,2.406667,1261.4,49141.4,46.28,291.333333,1.586667,6.226667,7.80,15.726667,12808.882353,2.246667
9916,100.033333,2.396667,1301.0,56111.9,41.25,312.133333,2.110000,5.286667,9.14,26.666667,13106.132992,3.173333


In [15]:
y_train = nndf['median_weekly_rent']
X_train = nndf.drop(['median_weekly_rent'], axis=1)

In [17]:
lr = LinearRegression()

In [18]:
lr.fit(X_train, y_train)

LinearRegression()

In [26]:
for feature, coef in zip(X_train.columns, lr.coef_):
    print(f'{feature}: {coef}')

num_homeless_persons: -0.0452986586026041
avg_household_size: 25.541400230908213
num_businesses: 0.010458598272182762
median_income: 0.004117903850460181
median_age: -0.8823015260924129
percent_aboriginal_torres_strait_islander: -12.754771895241383
percent_au_citizen: 1.8592489022789884
percent_overseas_born: 2.4845646176087297
percent_rental_properties: 0.21205270555018924
population: 0.008180188591528026
percent_unemployed: -11.740845344934339


In [30]:
clf = linear_model.Lasso(alpha=1)
clf.fit(X_train, y_train)

Lasso(alpha=1)

In [31]:
for feature, coef in zip(X_train.columns, clf.coef_):
    print(f'{feature}: {coef}')

num_homeless_persons: -0.04426637704739744
avg_household_size: 5.2221798401600354
num_businesses: 0.009907595164306698
median_income: 0.004234491380378563
median_age: -1.2800753007391346
percent_aboriginal_torres_strait_islander: -10.637936252072802
percent_au_citizen: 1.5761704988376906
percent_overseas_born: 2.696941107615436
percent_rental_properties: -0.21991930050946076
population: 0.007610639090695375
percent_unemployed: -11.335592798508053


In [42]:
future_years = [str(year) for year in range(2025, 2030)]

# Create a list that repeats each SA2 name for every year
future_sa2_column = SA2_names * len(future_years)

# Create a list that repeats each year for all SA2 names
future_year_column = future_years * len(SA2_names)

# Create the DataFrame
future_df = pd.DataFrame({
    'SA2_NAME21': future_sa2_column,
    'year': future_year_column
})

In [43]:
future_ndf = add_data(future_df)

In [81]:
future_nndf = future_ndf.drop(['housing_index', 'cpi_without_housing', 'median_weekly_rent'], axis=1)

In [82]:
future_nndf

,SA2_NAME21,year,num_homeless_persons,avg_household_size,num_businesses,median_income,median_age,percent_aboriginal_torres_strait_islander,percent_au_citizen,percent_overseas_born,percent_rental_properties,population,percent_unemployed
0,Alfredton,2025,56.781784,2.800000,1430.8,62670.3,35.62,1.293333,2.966667,18.42,32.093333,14420.682864,3.883333
1,Ballarat,2026,58.051643,2.166667,2852.5,61209.0,48.54,1.066667,4.833333,15.40,33.866667,14624.772379,4.433333
2,Buninyong,2027,59.333333,2.656667,603.6,65994.6,46.41,1.383333,5.216667,9.62,15.413333,14820.286445,3.063333
3,Delacombe,2028,50.000000,2.513333,1386.5,66293.6,32.18,2.726667,3.413333,15.66,28.680000,15014.808184,4.920000
4,Smythes Creek,2029,8.166667,3.000000,434.4,71381.0,43.69,2.800000,0.000000,8.02,3.640000,15208.063939,2.730000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,Otway,2025,29.200000,2.043333,564.6,41893.4,54.04,1.413333,11.200000,15.78,18.953333,14420.682864,3.456667
2606,Moyne - East,2026,35.333333,2.466667,1307.3,58866.4,43.26,2.066667,7.933333,7.40,15.900000,14624.772379,2.433333
2607,Moyne - West,2027,11.633333,2.356667,1483.4,55781.9,48.78,1.936667,6.526667,7.80,15.526667,14820.286445,1.396667
2608,Warrnambool - North,2028,110.533333,2.346667,1571.5,65271.4,44.60,2.260000,5.086667,10.24,25.666667,15014.808184,2.273333


In [83]:
X_test = future_nndf.drop(['SA2_NAME21', 'year'], axis=1)

In [84]:
y_pred = clf.predict(X_test)

In [85]:
future_nndf['predicted_median_rent'] = y_pred

In [89]:
future_nndf['year'] = future_nndf['year'].astype(int)
future_nndf.sort_values(by=['SA2_NAME21', 'year'], inplace=True)
future_nndf['pct_change_rent'] = future_nndf.groupby('SA2_NAME21')['predicted_median_rent'].pct_change() * 100
future_nndf['pct_change_rent'] = future_nndf['pct_change_rent'].astype(float)

future_nndf['pct_change_rent'] = future_nndf['pct_change_rent'].astype(float)

# Calculate the average percentage change for each SA2_NAME21
average_pct_change = future_nndf.groupby('SA2_NAME21')['pct_change_rent'].mean().reset_index()

# Rename the column to reflect it is an average
average_pct_change.rename(columns={'pct_change_rent': 'average_pct_change'}, inplace=True)

# Sort the average percentage change in descending order
sorted_avg_pct_change = average_pct_change.sort_values(by='average_pct_change', ascending=False)

In [93]:
sorted_avg_pct_change.tail(20)

,SA2_NAME21,average_pct_change
517,Yarram,1.118274
151,Donvale - Park Orchards,1.103300
436,St Albans - South,1.084024
239,Keilor Downs,1.077522
120,Craigieburn - Central,1.075215
454,Sydenham,1.064593
122,Craigieburn - North West,1.025268
167,Epping (Vic.) - West,1.023019
280,Malvern East,0.980096
229,Hoppers Crossing - South,0.977843
